# Poner todo junto

Hasta ahora hemos hecho lo siguiente en los cuadernos anteriores:

- **Cuadernos 01**: Cargamos el Azure Search Engine con PDFs enriquecidos en index: «cogsrch-index-files»
- **Cuaderno 02**: Añadimos modelos GPT de AzureOpenAI para mejorar la producción de la respuesta mediante el uso de Utility Chains de LLMs.
- **Cuaderno 03**: Añadimos memoria a nuestro sistema para alimentar un Chat Bot conversacional
- **Cuaderno 04**: Introducimos Agentes y Herramientas y construimos la primera Habilidad/Agente, que puede hacer RAG sobre un motor de búsqueda

Nos falta una cosa más: **¿Cómo unimos todas estas características en un inteligente GPT Smart Search Engine Chat Bot?

Queremos un asistente virtual para nuestra empresa que pueda recibir la pregunta, pensar qué herramienta utilizar y, a continuación, obtener la respuesta. El objetivo es que, independientemente de la fuente de la información, el Asistente pueda responder a la pregunta correctamente utilizando la herramienta adecuada.

En este Cuaderno vamos a crear ese Agente «inteligente» (también llamado Agente Maestro), que

1) entiende la pregunta, interactúa con el usuario 
2) habla con otros Agentes especializados
3) una vez que obtiene la respuesta, la entrega al usuario o deja que el Agente especializado la entregue directamente.

Este es el mismo concepto de [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agentes que hablan entre sí.


![image](https://www.microsoft.com/en-us/research/uploads/prod/2023/09/AutoGen_Fig1.png)

In [ ]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [ ]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Obtener las Herramientas - Agente DocSearch

**Considere el siguiente concepto:** Los agentes, que son esencialmente entidades de software diseñadas para realizar tareas específicas, pueden estar equipados con herramientas. Estas herramientas a su vez pueden ser otros agentes, cada uno poseyendo su propio conjunto de herramientas. Esto crea una estructura en capas en la que las herramientas pueden ir desde secuencias de código hasta acciones humanas, formando cadenas interconectadas. En última instancia, estás construyendo una red de agentes y sus respectivas herramientas, todos trabajando en colaboración para resolver una tarea específica (esto es lo que es ChatGPT). Esta red funciona aprovechando las capacidades únicas de cada agente y herramienta, creando un sistema dinámico y eficiente para la resolución de tareas.

En el fichero `common/utils.py` creamos Clases de Herramientas de Agente para cada una de las Funcionalidades que desarrollamos en Cuadernos anteriores. 


In [ ]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


In [ ]:
doc_indexes = ["cogsrch-index-files"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           callback_manager=cb_manager, verbose=False)

### Variables/perillas a utilizar para la personalización

Como has visto hasta ahora, hay muchas perillas que puedes marcar hacia arriba o hacia abajo con el fin de cambiar el comportamiento de tu aplicación GPT Smart Search engine, estas son las variables que puedes afinar:

- <u>llm</u>:
  - **deplyment_name**: este es el nombre de despliegue de su modelo Azure OpenAI. Esto, por supuesto, dicta el nivel de razonamiento y la cantidad de tokens disponibles para la conversación. Para un sistema de producción necesitará gpt-4-32k. Este es el modelo que te dará suficiente poder de razonamiento para trabajar con agentes, y suficientes tokens para trabajar con respuestas detalladas y memoria de conversación.
  - **temperature**: Cómo de creativas quieres que sean tus respuestas
  - **max_tokens**: Cómo de largas quieres que sean tus respuestas. Se recomienda un mínimo de 500
- <u>Herramientas</u>: A cada herramienta puedes agregar los siguientes parámetros para modificar los predeterminados (establecidos en utils.py), estos son muy importantes ya que forman parte del prompt del sistema y determina que herramienta usar y cuando.
  - **name**: el nombre de la herramienta
  - **description**: cuando el agente cerebral debe usar esta herramienta
- <u>DocSearchAgent</u>: 
  - **k**: Los k mejores resultados por índice de la acción de búsqueda de texto
  - **similarity_k**: los k mejores resultados combinados de la acción de búsqueda vectorial
  - **reranker_th**: umbral del reranker de búsqueda semántica. Selecciona los resultados que están por encima del umbral. Puntuación máxima posible=4

  
en `utils.py` también se puede afinar:
- <u>model_tokens_limit</u>: En esta función puedes editar cual es el máximo permitido de tokens reservados para el contenido. Recuerda que los restantes serán para el prompt del sistema más la respuesta

### Pruebe las herramientas

In [ ]:
# Test the Documents Search Tool with a question we know it doesn't have the knowledge for
printmd(doc_search.run("what is the weather today in Dallas?"))

In [ ]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(await doc_search.arun("What are some examples of reinforcement learning?"))

### Definir qué herramientas le vamos a dar a nuestro agente cerebral

Vaya a `common/utils.py` para verificar la definición de herramientas y las instrucciones sobre qué herramienta usar y cuándo

In [ ]:
tools = [doc_search ]

# Opción 1: usar funciones OpenAI como enrutador

Necesitamos un método para dirigir la pregunta a la herramienta correcta; una forma sencilla de hacerlo es utilizar las funciones de los modelos OpenAI a través de la API de herramientas (modelos 1106 y posteriores). Para hacer esto, necesitamos vincular estas herramientas/funciones al modelo y dejar que el modelo responda con la herramienta adecuada para usar.

La ventaja de esta opción es que no hay ningún otro agente en el medio entre los expertos (herramientas de agentes) y el usuario. Cada herramienta de agente responde directamente. Además, otra ventaja es que se pueden llamar varias herramientas en paralelo.

**Nota**: en este método es importante que cada herramienta de agente tenga el mismo mensaje de perfil del sistema para que cumplan con las mismas pautas de respuesta.

In [ ]:
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

In [ ]:
def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["type"]]
    return RunnablePassthrough.assign(output=itemgetter("args") | tool)

def print_response(result: List):
    for answer in result:
        printmd("**"+answer["type"] + "**" + ": " + answer["output"])
        printmd("----")
    
# .map() allows us to apply a function to a list of inputs.
call_tool_list = RunnableLambda(call_tool).map()
agent = llm_with_tools | JsonOutputToolsParser() | call_tool_list

In [ ]:
result = agent.invoke("hi, how are you, what is your name?")
print_response(result)

# Opción 2: utilizar un agente orientado al usuario que llame a los expertos en herramientas del agente

Con este método, creamos un agente de cara al usuario que habla con el usuario y también habla con los expertos (herramientas del agente).

### Inicializar el agente

In [ ]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [ ]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [ ]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [ ]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

### Hablemos ahora con nuestro chatbot de GPT Smart Search Engine

In [ ]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(brain_agent_executor.invoke({"question": "Hi, I'm Pablo Marin, how are you doing today?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "docsearch, what is a NP-complete problem?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "can you tell an example?"}, config=config)["output"])

# Resumen

¡Genial! ¡Acabamos de crear el motor de búsqueda inteligente GPT!
En este Cuaderno creamos el cerebro, el Agente de toma de decisiones que decide qué Herramienta utilizar para responder la pregunta del usuario. Esto es lo que era necesario para tener un chatbot inteligente.

Podemos tener muchas herramientas para realizar diferentes tareas, incluida la conexión a API, el manejo de sistemas de archivos e incluso el uso de humanos como herramientas. Para obtener más referencia, consulte [AQUÍ](https://python.langchain.com/docs/integrations/tools/)